# Construcción de una Base de Datos Vectorial desde cero

Una empresa llamada DataFind quiere crear su propia base de datos vectorial interna, sin depender de servicios externos.
Necesitan almacenar, indexar y buscar información basada en significado, no solo por coincidencia de texto.

Tu misión es simular una BDV completa capaz de:

Almacenar embeddings.

Buscar por similitud.

Implementar filtros por metadatos.

Medir la eficiencia y calidad de los resultados.

## Objetivos de la práctica

1. Generar embeddings simulados

- Cargar csv documentos_datafind.csv.

- Convertirlos en vectores de alta dimensión (usando numpy para simular embeddings).

2. Implementar componentes internos de la BDV:

- Storage Engine: guardar los vectores y metadatos.

- Index Builder: crear índices con estrategias de agrupamiento (simular IVF, HNSW o PQ).

- Query Engine: calcular similitudes (cosine y euclidean).

- Metadata Store: almacenar texto original, etiquetas o categorías.

3. Implementar consultas vectoriales

- Permitir que el usuario escriba una consulta y el sistema devuelva los documentos más similares.

- Simular embeddings de la consulta con el mismo método.

4. Agregar filtrado por metadatos

- Permitir buscar solo en documentos de una categoría (ej. “Tecnología”, “Salud”, “Educación”).

5. Evaluar el rendimiento y calidad de la búsqueda:

- Calcular tiempo de consulta.

- Evaluar qué métrica (cosine o euclidean) devuelve resultados más relevantes.

6. Simular optimización del índice:

- Comparar tiempos de búsqueda con distintas estrategias (sin índice, con IVF, con HNSW).

- Analizar cuál ofrece el mejor equilibrio entre velocidad y precisión.

7. Generar un informe final automático:

- Guardar resultados de las consultas, métricas y tiempos en un archivo reporte_resultados.csv.